In [14]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from data_collection import training_set
from torch.utils.data import DataLoader
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [15]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [19]:
encoded_input = tokenizer("I am testing", return_tensors="pt")
print(encoded_input)
gpt2_model(input_ids, attention_mask)
generated = gpt2_model.generate(**encoded_input, temperature=.9, do_sample=True)
print(tokenizer.batch_decode(generated))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'input_ids': tensor([[  40,  716, 4856]]), 'attention_mask': tensor([[1, 1, 1]])}


/home/linulun/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['I am testing it and I can think of quite a few users who use the Windows 7 installer.']


In [24]:
seq = torch.tensor(encoded_input["input_ids"])
output = gpt2_model(seq, labels=seq)
loss, logits = output[:2]

print(loss)
print(logits)
print(loss.shape)
print(logits.shape)

/tmp/ipykernel_8473/224967916.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq = torch.tensor(encoded_input["input_ids"])


tensor(7.6202, grad_fn=<NllLossBackward0>)
tensor([[[ -39.3084,  -39.0101,  -41.8375,  ...,  -46.9338,  -44.9074,
           -39.5150],
         [-114.8939, -114.2495, -119.9518,  ..., -122.6206, -121.7605,
          -117.6996],
         [ -94.5557,  -95.3818, -101.0136,  ..., -103.2835, -105.8857,
           -97.4768]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([])
torch.Size([1, 3, 50257])


In [25]:
print(gpt2_model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [16]:
train_dataset = training_set(["romantic", "friendly", "fight"], tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

Found cached dataset parquet (/home/linulun/.cache/huggingface/datasets/AlekseyKorshuk___parquet/AlekseyKorshuk--synthetic-romantic-characters-3b16d8e672467bfe/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/home/linulun/.cache/huggingface/datasets/AlekseyKorshuk___parquet/AlekseyKorshuk--synthetic-friendly-characters-8195740b6ede92c1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/home/linulun/.cache/huggingface/datasets/AlekseyKorshuk___parquet/AlekseyKorshuk--synthetic-fight-characters-dbee9baf48903647/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
print(train_dataset[0].shape)
print(train_dataset[1].shape)
print(train_dataset[2].shape)
print([[9], torch.tensor([0])])
print([0] * 10)

torch.Size([464])
torch.Size([413])
torch.Size([270])
[[9], tensor([0])]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
for param in gpt2_model.transformer.parameters():
    param.requires_grad = False


optimizer = torch.optim.Adam(gpt2_model.parameters(), lr=0.0001)

epochs = 10
epoch_losses = []

for _ in tqdm.tqdm(range(epochs)):
    epoch_loss = 0
    for sequence in train_loader:
        optimizer.zero_grad()   
        

        loss, logits = gpt2_model(sequence, labels=sequence)[:2]
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    epoch_losses.append(epoch_loss)


plt.plot(np.arange(epochs), epoch_losses)
plt.show()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [435] at entry 0 and [283] at entry 1